# Sequence models in natural language processing

Text mining course

## Outline

This document contains following contents.

- How to estimate parameters in a first-order HMM tagger
- How to use nltk.tag.hmm to train a first-order HMM tagger
- How to use CRF++ toolkit for POS tagging

## Implementation of a first-order HMM tagger

In a HMM, we need to to estimate emission and transition probabilties from a training data set. We will use the training data in the file `wikientrain.norm_pos`.

The file contains sentences whose words are associated with POS tags. 

In [40]:
# Change the path in your environment
TRAINING_DATA = "./data/POS/wiki-en-train.norm_pos"
TEST_DATA = "./data/POS/wiki-en-test.norm_pos"
TEST_RAW_DATA = "./data/POS/wiki-en-test.norm"
GOLD_TEST_DATA = "./data/POS/wiki-en-test.pos"

Now we load the data from the file into a list of strings and look at some first lines in the file

In [5]:
lines = []
with open(TRAINING_DATA, "r") as f:
    for line in f:
        line = line.strip() # strip trailing spaces and new-line char
        if line == "":
            continue
        lines.append(line)
print("# Loaded {} lines".format(len(lines)))
# Print first 5 lines
for i in range(5):
    print(lines[i])

# Loaded 1301 lines
Natural_JJ language_NN processing_NN -LRB-_-LRB- NLP_NN -RRB-_-RRB- is_VBZ a_DT field_NN of_IN computer_NN science_NN ,_, artificial_JJ intelligence_NN -LRB-_-LRB- also_RB called_VBN machine_NN learning_NN -RRB-_-RRB- ,_, and_CC linguistics_NNS concerned_VBN with_IN the_DT interactions_NNS between_IN computers_NNS and_CC human_JJ -LRB-_-LRB- natural_JJ -RRB-_-RRB- languages_NNS ._.
Specifically_RB ,_, it_PRP is_VBZ the_DT process_NN of_IN a_DT computer_NN extracting_VBG meaningful_JJ information_NN from_IN natural_JJ language_NN input_NN and\/or_CC producing_VBG natural_JJ language_NN output_NN ._.
In_IN theory_NN ,_, natural_JJ language_NN processing_NN is_VBZ a_DT very_RB attractive_JJ method_NN of_IN human_JJ --_: computer_NN interaction_NN ._.
Natural_JJ language_NN understanding_NN is_VBZ sometimes_RB referred_VBN to_TO as_IN an_DT AI-complete_JJ problem_NN because_IN it_PRP seems_VBZ to_TO require_VB extensive_JJ knowledge_NN about_IN the_DT outside_JJ world_N

Each line in the train file is a list of words with their POS tags. We will use that data to train an HMM tagger.

### Parameter estimation

Transition probabilities in a first-order HMM tagger are calculated as follows.

$$
P(t_i|t_{i-1})=\frac{C(t_{i-1},t_i)}{C(t_{i-1})}
$$

Here $C(t_{i-1},t_i)$ is the number of times the tag $t_i$ follows the tag $t_{i-1}$ in the corpus and $C(t_{i-1})$ is the number of times tag $t_{i-1}$ occurs in the corpus.

Emission probabilities are calculated as follows

$$
P(w_i|t_i)=\frac{C(t_i,w_i)}{C(t_i)}
$$

$C(t_i,w_i)$ is the number of times the word $w_i$ is associated with the tag $t_i$

Below is the implementation of transition probability estimation. Students need to implement emission probability estimation by themselves.

In [22]:
from collections import defaultdict

def train_hmm():
    transition = {}  # Transition probabilities
    emis = {} # Emission probabilities that students need to estimate
    transition_count = defaultdict(int)  # Transition counts 
    context = defaultdict(int)  # Count of the context
    
    with open(TRAINING_DATA, "r") as f:
        for line in f:
            line = line.strip()
            if line == "":
                continue
            previous = "<s>"  # Make the sentence start
            context[previous] += 1
            wordtags = line.split()
            for wordtag in wordtags:
                word, tag = wordtag.split("_")
                transition_count[(previous + " " + tag)] += 1
                context[tag] += 1
                previous = tag
            transition_count[previous + " " + "</s>"] += 1
    # Calculate transition probabilities and print out
    # In real program, you should not print probabilities out
    for key, count in transition_count.items():
        previous, tag = key.split(" ")
        transition[key] = count/context[previous]
        print("T {} {}".format(key, transition[key]))
    # Calculate emission probabilities
    # Save probabilities into a model file

In [23]:
train_hmm()

T <s> JJ 0.1083781706379708
T JJ NN 0.4236357765579593
T NN NN 0.16897106109324758
T NN -LRB- 0.02652733118971061
T -LRB- NN 0.23035230352303523
T NN -RRB- 0.02508038585209003
T -RRB- VBZ 0.04878048780487805
T VBZ DT 0.24807903402854006
T DT NN 0.5120500782472613
T NN IN 0.20562700964630226
T IN NN 0.1861437908496732
T NN , 0.10868167202572347
T , JJ 0.0892756878158338
T -LRB- RB 0.06233062330623306
T RB VBN 0.13747954173486088
T VBN NN 0.08943862987630828
T -RRB- , 0.21138211382113822
T , CC 0.15553060078607525
T CC NNS 0.08131655372700872
T NNS VBN 0.026446914526638558
T VBN IN 0.4196003805899144
T IN DT 0.33908496732026144
T DT NNS 0.07918622848200313
T NNS IN 0.21885779992334228
T IN NNS 0.07973856209150326
T NNS CC 0.05596013798390188
T CC JJ 0.1442400774443369
T JJ -LRB- 0.006780755569906361
T -LRB- JJ 0.06775067750677506
T JJ -RRB- 0.0038747174685179204
T -RRB- NNS 0.02168021680216802
T NNS . 0.13491759294748945
T . </s> 0.9785768936495792
T <s> RB 0.08993082244427364
T RB , 0.1

After training HMM tagger, you need to save transition and emission probabilities into a model file using above format. You may want to use "E" for marking emission probabilties.

### Viterbi algorithm

Implementation of Viterbi algorithm in HMM tagger is left as the assignment for students.

Students need to define following function.

In [ ]:
def test_hmm():
    # Implementation of Viterbi algorithm
    pass

## Using nltk.tag.hmm for training a POS tagger

In this section, we will use the module `nltk.tag.hmm` to train a POS tagger.

In [49]:
# Load training data into list of sentences. Each sentence is a list of tuples (word, pos)
train_data = []

def load_data(input_file):    
    data = []
    with open(input_file, "r") as f:
        for line in f:
            line = line.strip()
            if line == "":
                continue
            wordtags = line.split()
            sentence = []
            for wordtag in wordtags:
                word, tag = wordtag.split("_")
                sentence.append((word,tag))
            data.append(sentence)
    return data

train_data = load_data(TRAINING_DATA)
print("# Loaded {} sentences".format(len(train_data)))
print("# Show the first sentence")
print(train_data[0])


# Loaded 1301 sentences
# Show the first sentence
[('Natural', 'JJ'), ('language', 'NN'), ('processing', 'NN'), ('-LRB-', '-LRB-'), ('NLP', 'NN'), ('-RRB-', '-RRB-'), ('is', 'VBZ'), ('a', 'DT'), ('field', 'NN'), ('of', 'IN'), ('computer', 'NN'), ('science', 'NN'), (',', ','), ('artificial', 'JJ'), ('intelligence', 'NN'), ('-LRB-', '-LRB-'), ('also', 'RB'), ('called', 'VBN'), ('machine', 'NN'), ('learning', 'NN'), ('-RRB-', '-RRB-'), (',', ','), ('and', 'CC'), ('linguistics', 'NNS'), ('concerned', 'VBN'), ('with', 'IN'), ('the', 'DT'), ('interactions', 'NNS'), ('between', 'IN'), ('computers', 'NNS'), ('and', 'CC'), ('human', 'JJ'), ('-LRB-', '-LRB-'), ('natural', 'JJ'), ('-RRB-', '-RRB-'), ('languages', 'NNS'), ('.', '.')]


Next will will use the training data to train a HMM tagger

In [56]:
# Import the toolkit and tags
import nltk
from nltk.tag import hmm

# Setup a trainer with default(None) values
# And train with the data
trainer = hmm.HiddenMarkovModelTrainer()
tagger = trainer.train_supervised(train_data)
print(tagger)

gold_sentences = load_data(TEST_DATA)
print("# Test accuracy: {}".format(tagger.evaluate(gold_sentences)))


<HiddenMarkovModelTagger 42 states and 5233 output symbols>
# Test accuracy: 0.35261889108042954


Now we will use the trained tagger to tag some sentences in the test data set.

In [33]:
print(tagger.tag("The bass line of the song is too weak .".split()))

print(tagger.tag("Still , supervised systems continue to perform best .".split()))

print(tagger.tag("A set of testing words .".split()))

[('The', 'DT'), ('bass', 'JJ'), ('line', 'JJ'), ('of', 'JJ'), ('the', 'JJ'), ('song', 'JJ'), ('is', 'JJ'), ('too', 'JJ'), ('weak', 'JJ'), ('.', 'JJ')]
[('Still', 'JJ'), (',', 'JJ'), ('supervised', 'JJ'), ('systems', 'JJ'), ('continue', 'JJ'), ('to', 'JJ'), ('perform', 'JJ'), ('best', 'JJ'), ('.', 'JJ')]
[('A', 'DT'), ('set', 'NN'), ('of', 'IN'), ('testing', 'NN'), ('words', 'NNS'), ('.', '.')]


Now we use the trained HMM tagger to tag sentences in the test data and write the output to a file says `nltk_answer.pos`

In [35]:
output_file = "./nltk_answer.pos"
fo = open(output_file, "w")
with open(TEST_RAW_DATA, "r") as f:
    for line in f:
        line = line.strip()
        if line == "":
            continue
        tagged_sen = tagger.tag(line.split())
        tags = [ tag for _, tag in tagged_sen]
        fo.write("{}\n".format(" ".join(tags)))
fo.close()
            

Now we use the perl script to evaluate the result of the tagger trained by `nltk.tag.hmm`

    ./gradepos.pl ./data/POS/wiki-en-test.pos ./nltk_answer.pos

It turned out that the trained hmm tagger is not good.

```
Accuracy: 35.26% (1609/4563)

Most common mistakes:
NN --> JJ	607
IN --> JJ	358
NNS --> JJ	281
DT --> JJ	250
. --> JJ	152
, --> JJ	152
RB --> JJ	125
VBN --> JJ	115
CC --> JJ	107
VB --> JJ	80
```

You can customize the HMM tagger by using options in training. See the the source code of `nltk.tag.hmm` in [https://www.nltk.org/_modules/nltk/tag/hmm.html](https://www.nltk.org/_modules/nltk/tag/hmm.html) and the documentation in [https://www.nltk.org/api/nltk.tag.html?highlight=tag%20hmm#module-nltk.tag.hmm](https://www.nltk.org/api/nltk.tag.html?highlight=tag%20hmm#module-nltk.tag.hmm)

## Use nltk.tag.crf for training an POS tagger

This section will use `nltk.tag.crf` module to train and test an POS tagger

Reference: [https://www.nltk.org/_modules/nltk/tag/crf.html](https://www.nltk.org/_modules/nltk/tag/crf.html)


In [51]:
from nltk.tag import CRFTagger

crf_model_file = "model.crf.tagger"
ct = CRFTagger()
ct.train(train_data, crf_model_file)


Now we learn how to load the crf model file and evaluate on the test data

In [52]:
from nltk.tag import CRFTagger

ct = CRFTagger()
crf_model_file = "model.crf.tagger"
ct.set_model_file(crf_model_file)

gold_sentences = load_data(TEST_DATA)
print("# Test accuracy: {}".format(ct.evaluate(gold_sentences)))

# Test accuracy: 0.9289940828402367


We obtained much higher accuracy than the HMM tagger trained by using `nltk.tag.hmm`.

## Use CRF++ for POS tagging

In this section, we will learn how to use CRF++ ([https://taku910.github.io/crfpp/](https://taku910.github.io/crfpp/)) for POS tagging.

You should download and install CRF++ first.

### Building training/test file for CRF++

We convert data files into format that is used by CRF++.

In [46]:
def convert_data(input_file, output_file):
    with open(output_file, "w") as fo:
        with open(input_file, "r") as f:
            for line in f:
                line = line.strip()
                if line == "":
                    continue
                wordtags = line.split()
                for wordtag in wordtags:
                    word, tag = wordtag.split("_")
                    fo.write("{} {}\n".format(word, tag))
                fo.write("\n")

Now, we use above functions to make CRF++ training and test data

In [47]:
crf_train = "./train_crf.txt"
crf_test = "./test_crf.txt"

convert_data(TRAINING_DATA, crf_train)

convert_data(TEST_DATA, crf_test)

In order to train a CRF model, we need to create a template file. We already prepare the template file for POS tagging task as follows.

```
# Unigram
U00:%x[-2,0]
U01:%x[-1,0]
U02:%x[0,0]
U03:%x[1,0]
U04:%x[2,0]
U05:%x[-1,0]/%x[0,0]
U06:%x[0,0]/%x[1,0]

# Bigram
B
```

Now we train CRF++ model by using following command line

    crf_learn -p 2 template train_crf.txt crfpp_model

Now test on the test data

    crf_test -m crfpp_model ./test_crf.txt > ./test_crf.out.txt

We convert back to POS file for evaluation

In [59]:
import re

crf_output = "./test_crfpp.pos"

def write_pos_file(pos_output, crf_output):
    data = []
    sentence = []
    with open(crf_output, "r") as f:
        for line in f:
            # Blank line
            if re.search(r"^[\s\t\n]*$", line):
                if len(sentence) > 0:
                    data.append(sentence)
                    sentence = []
                else:
                    continue
            else:
                line = line.strip()
                word, tag, predicted_tag = line.split()
                sentence.append(predicted_tag)
    with open(pos_output, "w") as fo:
        for sentence in data:
            fo.write("{}\n".format(" ".join(sentence)))

write_pos_file(crf_output, "./test_crf.out.txt")

Now we can use the perl script `gradepos.pl` for evaluation.

## Summary

In this document, we have learned:

- How to estimate parameters for an HMM tagger
- Using `nltk.tag.hmm` and `ntlk.tag.crf` for building POS taggers
- Using CRF++ for building CRF POS tagger